1. 부처님께 기도 드리고 시작 

In [1]:
a=(
    '''
//                   _ooOoo_
//                  o8888888o
//                  88" . "88
//                  (| -_- |)
//                  O\  =  /O
//               ____/`---'\____
//             .'  \\|     |//  `.
//            /  \\|||  :  |||//  \\
//           /  _||||| -:- |||||-  \\
//           |   | \\\  -  /// |   |
//           | \_|  ''\---/''  |   |
//           \  .-\__  `-`  ___/-. /
//         ___`. .'  /--.--\  `. . __
//      ."" '<  `.___\_<|>_/___.'  >'"".
//     | | :  `- \`.;`\ _ /`;.`/ - ` : | |
//     \  \ `-.   \_ __\ /__ _/   .-` /  /
//======`-.____`-.___\_____/___.-`____.-'======
//                   `=---='
//
//^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
//      Buddah Bless        NO BUGS
//         God Bless        Never Crash
'''
)

In [3]:
print(a)


//                   _ooOoo_
//                  o8888888o
//                  88" . "88
//                  (| -_- |)
//                  O\  =  /O
//               ____/`---'\____
//             .'  \|     |//  `.
//            /  \|||  :  |||//  \
//           /  _||||| -:- |||||-  \
//           |   | \\  -  /// |   |
//           | \_|  ''\---/''  |   |
//           \  .-\__  `-`  ___/-. /
//         ___`. .'  /--.--\  `. . __
//      ."" '<  `.___\_<|>_/___.'  >'"".
//     | | :  `- \`.;`\ _ /`;.`/ - ` : | |
//     \  \ `-.   \_ __\ /__ _/   .-` /  /
//======`-.____`-.___\_____/___.-`____.-'======
//                   `=---='
//
//^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
//      Buddah Bless        NO BUGS
//         God Bless        Never Crash



2. mediapipe로 delete 추가하기 
- gesture 넣기 (dict으로)
- MediaPipe Model 
- 

In [ ]:
import cv2 # 웹캠 제어 및 ML 사용 
import numpy as np # 배열
import keyboard 
import time # 시간 
import mediapipe as mp # 손 인식 
from sklearn import neighbors # knn 

max_num_hands=1 # 최대 손 인식: 1 
gesture = {
    0:'a',1:'b',2:'c',3:'d',4:'e',5:'f',6:'g',7:'h',
    8:'i',9:'j',10:'k',11:'l',12:'m',13:'n',14:'o',
    15:'p',16:'q',17:'r',18:'s',19:'t',20:'u',21:'v',
    22:'w',23:'x',24:'y',25:'z',26:'spacing',27:'clear',28:'delete'
}

# MediaPipe hands model 
mp_hands=mp.solutions.hands # 웹캠 손 위치 정보 유틸리티
mp_drawing = mp.solutions.drawing_utils # 손 정보 기반으로 그리기 

# 손가락 detection 모듈을 초기화 
hands=mp_hands.Hands(
    max_num_hands=max_num_hands, # 최대 손 인식 
    min_detection_confidence=0.5, # 최소 탐지 신뢰도 기본값=0.5 
    min_tracking_confidence=0.5 # 최소 추적 신뢰도 기본값=0.5  
)

# 제스처 인식 모델 
f=open('test.txt','w') # 학습시킬 dataset 저장하기 
file=np.genfromtxt('dataSet.txt',delimiter=',')
angle = file[:,:-1].astype(np.float32) # 각도
label = file[:, -1].astype(np.float32) # 라벨
knn=cv2.ml.KNearest_create() # knn(k-최근접 알고리즘)으로  
knn.train(angle,cv2.ml.ROW_SAMPLE,label) # 학습!

# 웹캠 열기 
cap=cv2.VideoCapture(0) 

# 웹캠에서 한 프레임씩 이미지를 읽어옴 
startTime = time.time()
prev_index = 0
sentence = ''
recognlzeDelay = 1
while True:
    ret,img = cap.read()
    if not ret:
        break
    img = cv2.flip(img, 1) 
    imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    result = hands.process(imgRGB)

    # 각도를 인식하고 제스처를 인식하는 부분 
    if result.multi_hand_landmarks is not None: # 만약 손을 인식하면
            for res in result.multi_hand_landmarks: 
                # joint == 랜드마크에서 빨간 점
                # joint는 21개가 있고 x,y,z 좌표니까 21,3
                joint = np.zeros((21,3))
            
            for j,lm in enumerate(res.landmark):
                joint[j] = [lm.x,lm.y,lm.z] # 각 joint마다 x,y,z 좌표 저장  
            
            # Compute angles between joints joint마다 각도 계산 
            # **공식문서 들어가보면 각 joint 번호의 인덱스가 나옴**  
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:] 
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:] 

            v = v2 - v1 # [20,3]관절벡터  
            # v 정규화(식 축소, api가 간단해짐)
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            # joint(빨간점)을 통해 역삼각함수 
            # **내적(벡터 곱 종류) 후 arccos으로 각도를 구해줌** 
            compareV1 = v[[0,1,2,4,5,6,7,8,9,10,12,13,14,16,17],:]
            compareV2 = v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]
            angle = np.arccos(np.einsum('nt,nt->n',compareV1,compareV2))
            angle = np.degrees(angle)

            # a 누르면 저장하기 
            # 학습 모델을 만들어야 할때 사용  
            if keyboard.is_pressed('a'):
                for num in angle:
                    num = round(num,6)
                    f.write(str(num))
                    f.write(',')
                f.write('28.00000') # gesture에 들어갈 key 넣어주기 
                f.write('\n')
                print("next")
                print('---------------------')
            
            # 학습시킨 제스쳐 모델에 참조 
            # 참고: http://www.gisdeveloper.co.kr/?p=6973 
            data = np.array([angle], dtype=np.float32)
            ret, result, neighbors,dist = knn.findNearest(data,3)
            index = int(result[0][0])

            cv2.putText(img, text=gesture[index].upper(), org=(int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

            # Draw gesture result  
            if index in gesture.keys(): # 만약 index가 gesture.key에 있으면 
                # prev_index=0, startTime = time.time()
                if index != prev_index:
                    startTime = time.time()
                    prev_index = index
                else:
                    if time.time() - startTime > recognlzeDelay:
                        if index == 26: # spacing 
                            sentence += ' '
                        elif index == 27: # clear 
                            sentence = ''
                        elif index == 28: # delete 
                            sentence = sentence[:-1]
                        else:
                            sentence += gesture[index]
                        startTime = time.time()
            
            # 손에 랜드마크를 그려줌  
            mp_drawing.draw_landmarks(img,res,mp_hands.HAND_CONNECTIONS)
    # 화면에 글자 보이게 하기  
    cv2.putText(img,sentence,(20,440),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),3)
    cv2.imshow('HandTracking',img)

    # q키 누르면 중단하기 
    if cv2.waitKey(1) == ord('q'):
            break

3. flask로 영상 보여주기 

In [ ]:
from flask import Flask, Response, render_template
import cv2

app = Flask(__name__)
video = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier()
face_cascade.load(cv2.samples.findFile("static/haarcascade_frontalface_alt.xml"))

@app.route('/')
def index():
    return render_template("index.html")

@app.route('/about')
def about():
    return render_template("about.html")


def gen(video):
    while True:
        success, image = video.read()
        frame_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        frame_gray = cv2.equalizeHist(frame_gray)

        faces = face_cascade.detectMultiScale(frame_gray)

        for (x, y, w, h) in faces:
            center = (x + w//2, y + h//2)
            cv2.putText(image, "X: " + str(center[0]) + " Y: " + str(center[1]), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
            image = cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

            faceROI = frame_gray[y:y+h, x:x+w]
        ret, jpeg = cv2.imencode('.jpg', image)

        frame = jpeg.tobytes()
        
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n\r\n')

@app.route('/video_feed')
def video_feed():
    global video
    return Response(gen(video),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host='127.0.0.1', port=8572, threaded=True)

4. flask로 mediapipe 내용 넣기 

In [ ]:
# 참고1-가위바위보: https://url.kr/uy129r
# 참고2-수화: https://developeralice.tistory.com/12 

from flask import Flask, Response, render_template # flask 
import cv2 # 웹캠 제어 및 ML 사용 
import numpy as np # 배열
import keyboard 
import time 
import mediapipe as mp # 손 인식 
from sklearn import neighbors # knn 

app = Flask(__name__)

video = cv2.VideoCapture(0) # 웹캠 키기 
max_num_hands=1 # 최대 손 인식: 1  
gesture = {
    0:'a',1:'b',2:'c',3:'d',4:'e',5:'f',6:'g',7:'h',
    8:'i',9:'j',10:'k',11:'l',12:'m',13:'n',14:'o',
    15:'p',16:'q',17:'r',18:'s',19:'t',20:'u',21:'v',
    22:'w',23:'x',24:'y',25:'z',26:'spacing',27:'clear',28:'delete'
}

# MediaPipe hands model 
mp_hands=mp.solutions.hands  # 웹캠에서 손마디 및 포인트 그리기1
mp_drawing = mp.solutions.drawing_utils  # 웹캠에서 손마디 및 포인트 그리기2 

# 손가락 detection 모듈을 초기화 
hands=mp_hands.Hands(
    max_num_hands=max_num_hands, # 최대 손 인식 
    min_detection_confidence=0.5, # 0.5로 해두는 게 좋다! 
    min_tracking_confidence=0.5
)

# 제스처 인식 모델 
f=open('test.txt','w') # 학습시킬 dataset 저장하기 
file=np.genfromtxt('dataSet.txt',delimiter=',') # 학습시킨 모델 
angle = file[:,:-1].astype(np.float32) # 각도
label = file[:, -1].astype(np.float32) # 라벨
knn=cv2.ml.KNearest_create() # knn(k-최근접 알고리즘)으로  
knn.train(angle,cv2.ml.ROW_SAMPLE,label) # 학습!

@app.route('/')
def index():
    return render_template('index.html')

def gen(video):
    # 웹캠 열기 
    cap=cv2.VideoCapture(0) 

    # 웹캠에서 한 프레임씩 이미지를 읽어옴 
    startTime = time.time()
    prev_index = 0
    sentence = ''
    recognlzeDelay = 1
    while True:
        ret,img = cap.read()
        if not ret:
            break
        img = cv2.flip(img, 1) 
        imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        result = hands.process(imgRGB)

        # 각도를 인식하고 제스처를 인식하는 부분 
        if result.multi_hand_landmarks is not None: # 만약 손을 인식하면
            for res in result.multi_hand_landmarks: 
                # joint == 랜드마크에서 빨간 점
                # joint는 21개가 있고 x,y,z 좌표니까 21,3
                joint = np.zeros((21,3))
                
                for j,lm in enumerate(res.landmark):
                    joint[j] = [lm.x,lm.y,lm.z] # 각 joint마다 x,y,z 좌표 저장 
                
                # Compute angles between joints joint마다 각도 계산 
                # **공식문서 들어가보면 각 joint 번호의 인덱스가 나옴**
                v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:] # Parent joint 
                v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:] # Child joint 

                v = v2 - v1 # [20,3]관절벡터  
                # Normalize v 
                v = v / np.linalg.norm(v, axis=1)[:, np.newaxis] # 벡터 정규화(크기 1 벡터) = v / 벡터의 크기 

                # joint(빨간점)을 통해 역삼각함수 
                # **내적(벡터 곱 종류) 후 arccos으로 각도를 구해줌** 
                compareV1 = v[[0,1,2,4,5,6,7,8,9,10,12,13,14,16,17],:]
                compareV2 = v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]
                angle = np.arccos(np.einsum('nt,nt->n',compareV1,compareV2))

                # Convert radian to degree 
                angle = np.degrees(angle)

                # a 누르면 저장하기 
                # 학습 모델을 만들어야 할때 사용 
                if keyboard.is_pressed('a'):
                    for num in angle:
                        num = round(num,6)
                        f.write(str(num))
                        f.write(',')
                    f.write('28.00000') # gesture에 들어갈 key 넣어주기 
                    f.write('\n')
                    print("next")
                    print('---------------------')

                # 학습시킨 제스쳐 모델에 참조 
                # 참고: http://www.gisdeveloper.co.kr/?p=6973 
                data = np.array([angle], dtype=np.float32)
                ret, result, neighbors,dist = knn.findNearest(data,3) #
                index = int(result[0][0]) # 인덱스 저장 

                cv2.putText(img, text=gesture[index].upper(), org=(int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

                # Draw gesture result 
                if index in gesture.keys(): # 만약 index가 gesture.key에 있으면 
                    # prev_index=0, startTime = time.time() 
                    if index != prev_index: 
                        startTime = time.time()
                        prev_index = index 
                    else:
                        if time.time() - startTime > recognlzeDelay:
                            if index == 26: # spacing 
                                sentence += ' '
                            elif index == 27: # clear 
                                sentence = ''
                            elif index == 28: # delete 
                                sentence = sentence[:-1]
                            else: # 모두 아니면 알파벳 계속 나오기 
                                sentence += gesture[index]
                            startTime = time.time()

                # 손에 랜드마크를 그려줌 
                mp_drawing.draw_landmarks(img,res,mp_hands.HAND_CONNECTIONS)
        
        # 화면에 글자 보이게 하기 
        cv2.putText(img,sentence,(20,440),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),3)
        
        # q키 누르면 중단하기 
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

        ret, jpeg = cv2.imencode('.jpg', img)
        frame = jpeg.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n\r\n')

@app.route('/video_feed')
def video_feed():
    # gen함수 활용하여 웹캠 열기 
    # 참고-https://url.kr/uy129r 
    # return Resopnse-HTTP 예외를 쉽게 처리할 수 있도록 돕는 함수 
    global video
    return Response(gen(video),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/template')
def template():
    return render_template('template.html')

@app.route('/schedule')
# 노션 html에 접속 
# 노션 페이지에서 export 한 html 파일 
def schedule():
    return render_template('Task Progress List.html')

if __name__ == '__main__':
    app.run(host='localhost', port=8572, threaded=True,debug=True)
